# 推理引擎ONNX Runtime部署-预测单张图像

使用推理引擎 ONNX Runtime，读取 ONNX 格式的模型文件，对单张图像文件进行预测。

同济子豪兄 https://space.bilibili.com/1900783

2022-8-22 2023-5-8

## 应用场景

以下代码在需要部署的硬件上运行（本地PC、嵌入式开发板、树莓派、Jetson Nano、服务器）

只需把`onnx`模型文件发到部署硬件上，并安装 ONNX Runtime 环境，用下面几行代码就可以运行模型了。

## 导入工具包

In [1]:
import onnxruntime
import numpy as np
import torch
import torch.nn.functional as F

import pandas as pd

## 载入 onnx 模型，获取 ONNX Runtime 推理器

In [2]:
ort_session = onnxruntime.InferenceSession('resnet18_imagenet.onnx')

## 构造随机输入，获取输出结果

In [4]:
x = torch.randn(1, 3, 256, 256).numpy()

In [5]:
x.shape

(1, 3, 256, 256)

In [6]:
# onnx runtime 输入
ort_inputs = {'input': x}

# onnx runtime 输出
ort_output = ort_session.run(['output'], ort_inputs)[0]

注意，输入输出张量的名称需要和 torch.onnx.export 中设置的输入输出名对应

In [7]:
ort_output.shape

(1, 1000)

In [7]:
# ort_output

## 载入一张真正的测试图像

In [8]:
img_path = 'banana1.jpg'

In [9]:
# 用 pillow 载入
from PIL import Image
img_pil = Image.open(img_path)

In [11]:
# img_pil

## 预处理函数

In [12]:
from torchvision import transforms

# 测试集图像预处理-RCTN：缩放裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(256),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 运行预处理

In [13]:
input_img = test_transform(img_pil)

In [14]:
input_img.shape

torch.Size([3, 256, 256])

In [15]:
input_tensor = input_img.unsqueeze(0).numpy()

In [16]:
input_tensor.shape

(1, 3, 256, 256)

## 推理预测

In [17]:
# ONNX Runtime 输入
ort_inputs = {'input': input_tensor}

# ONNX Runtime 输出
pred_logits = ort_session.run(['output'], ort_inputs)[0]
pred_logits = torch.tensor(pred_logits)

In [18]:
pred_logits.shape

torch.Size([1, 1000])

In [19]:
# 对 logit 分数做 softmax 运算，得到置信度概率
pred_softmax = F.softmax(pred_logits, dim=1) 

In [20]:
pred_softmax.shape

torch.Size([1, 1000])

## 解析预测结果

In [22]:
# 取置信度最高的前 n 个结果
n = 3

In [23]:
top_n = torch.topk(pred_softmax, n)

In [24]:
top_n

torch.return_types.topk(
values=tensor([[9.9669e-01, 2.6005e-03, 3.0254e-04]]),
indices=tensor([[954, 939, 941]]))

In [25]:
# 预测类别
pred_ids = top_n.indices.numpy()[0]

In [26]:
pred_ids

array([954, 939, 941])

In [27]:
# 预测置信度
confs = top_n.values.numpy()[0]

In [28]:
confs

array([9.9668556e-01, 2.6005376e-03, 3.0253988e-04], dtype=float32)

## 载入类别 ID 和 类别名称 对应关系

In [38]:
df = pd.read_csv('imagenet_class_index.csv')
idx_to_labels = {}
for idx, row in df.iterrows():
    idx_to_labels[row['ID']] = row['class']   # 英文
#     idx_to_labels[row['ID']] = row['Chinese'] # 中文

In [39]:
# idx_to_labels

## 分别用英文和中文打印预测结果

In [40]:
for i in range(n):
    class_name = idx_to_labels[pred_ids[i]] # 获取类别名称
    confidence = confs[i] * 100             # 获取置信度
    text = '{:<20} {:>.3f}'.format(class_name, confidence)
    print(text)

banana               99.669
zucchini             0.260
acorn_squash         0.030
